# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return images
    #return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print (Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.4)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [ ]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 7.74446 Disc Loss: 1.76523 Q Losses: [0.21422461, 2.3141761]
epoch:0 batch_done:2 Gen Loss: 9.55653 Disc Loss: 0.341702 Q Losses: [0.1500565, 2.3029916]
epoch:0 batch_done:3 Gen Loss: 12.6012 Disc Loss: 0.313274 Q Losses: [0.17423716, 2.3220661]
epoch:0 batch_done:4 Gen Loss: 0.00561882 Disc Loss: 2.93934 Q Losses: [0.1517016, 2.2868762]
epoch:0 batch_done:5 Gen Loss: 4.84678 Disc Loss: 8.12348 Q Losses: [0.12837598, 2.2783859]
epoch:0 batch_done:6 Gen Loss: 6.71776 Disc Loss: 0.685103 Q Losses: [0.13120843, 2.2631927]
epoch:0 batch_done:7 Gen Loss: 1.74364 Disc Loss: 0.820542 Q Losses: [0.11917087, 2.2742789]
epoch:0 batch_done:8 Gen Loss: 9.83898 Disc Loss: 1.53457 Q Losses: [0.09216439, 2.2650433]
epoch:0 batch_done:9 Gen Loss: 8.10418 Disc Loss: 0.842016 Q Losses: [0.13054526, 2.2629499]
epoch:0 batch_done:10 Gen Loss: 2.73808 Disc Loss: 0.35335 Q Losses: [0.13370244, 2.2205045]
epoch:0 batch_done:11 Gen Loss: 11.0294 Disc Loss: 2.99439 Q Losses: [0.1

epoch:0 batch_done:89 Gen Loss: 6.7349 Disc Loss: 0.0194738 Q Losses: [0.053753067, 0.89239991]
epoch:0 batch_done:90 Gen Loss: 5.97347 Disc Loss: 0.0288662 Q Losses: [0.049722239, 0.86780715]
epoch:0 batch_done:91 Gen Loss: 6.63178 Disc Loss: 0.0157772 Q Losses: [0.061414968, 0.89548206]
epoch:0 batch_done:92 Gen Loss: 6.82764 Disc Loss: 0.00973621 Q Losses: [0.066409752, 0.8635416]
epoch:0 batch_done:93 Gen Loss: 6.34347 Disc Loss: 0.0139117 Q Losses: [0.051662244, 0.8470754]
epoch:0 batch_done:94 Gen Loss: 6.07166 Disc Loss: 0.0109578 Q Losses: [0.055166345, 0.85676813]
epoch:0 batch_done:95 Gen Loss: 5.50397 Disc Loss: 0.0136194 Q Losses: [0.048959114, 0.82888246]
epoch:0 batch_done:96 Gen Loss: 5.67049 Disc Loss: 0.0147191 Q Losses: [0.065827608, 0.82724422]
epoch:0 batch_done:97 Gen Loss: 6.90934 Disc Loss: 0.0217528 Q Losses: [0.049925599, 0.81955725]
epoch:0 batch_done:98 Gen Loss: 7.64283 Disc Loss: 0.0152874 Q Losses: [0.06489557, 0.80858171]
epoch:0 batch_done:99 Gen Loss: 7

epoch:0 batch_done:173 Gen Loss: 7.35984 Disc Loss: 0.0144048 Q Losses: [0.040627327, 0.37627664]
epoch:0 batch_done:174 Gen Loss: 7.50647 Disc Loss: 0.00751522 Q Losses: [0.039026313, 0.39565244]
epoch:0 batch_done:175 Gen Loss: 7.07023 Disc Loss: 0.00622706 Q Losses: [0.032066096, 0.37574685]
epoch:0 batch_done:176 Gen Loss: 5.89771 Disc Loss: 0.00651128 Q Losses: [0.043428622, 0.36005372]
epoch:0 batch_done:177 Gen Loss: 6.76559 Disc Loss: 0.00781545 Q Losses: [0.029346906, 0.36572695]
epoch:0 batch_done:178 Gen Loss: 7.34245 Disc Loss: 0.00458954 Q Losses: [0.033302169, 0.37619054]
epoch:0 batch_done:179 Gen Loss: 7.75165 Disc Loss: 0.00309449 Q Losses: [0.040099978, 0.36192822]
epoch:0 batch_done:180 Gen Loss: 8.22525 Disc Loss: 0.00186831 Q Losses: [0.039652787, 0.37560213]
epoch:0 batch_done:181 Gen Loss: 7.50628 Disc Loss: 0.00340773 Q Losses: [0.032970425, 0.3642157]
epoch:0 batch_done:182 Gen Loss: 6.4106 Disc Loss: 0.00731912 Q Losses: [0.034819309, 0.34256989]
epoch:0 batch

epoch:1 batch_done:51 Gen Loss: 10.0955 Disc Loss: 0.00106203 Q Losses: [0.016798833, 0.19960874]
epoch:1 batch_done:52 Gen Loss: 7.81025 Disc Loss: 0.00134414 Q Losses: [0.027156999, 0.198843]
epoch:1 batch_done:53 Gen Loss: 7.19889 Disc Loss: 0.00223747 Q Losses: [0.030534353, 0.18901207]
epoch:1 batch_done:54 Gen Loss: 6.11602 Disc Loss: 0.00485861 Q Losses: [0.020435795, 0.21045974]
epoch:1 batch_done:55 Gen Loss: 6.7622 Disc Loss: 0.00475306 Q Losses: [0.019869868, 0.19274004]
epoch:1 batch_done:56 Gen Loss: 4.11114 Disc Loss: 0.0221417 Q Losses: [0.016349107, 0.19144675]
epoch:1 batch_done:57 Gen Loss: 13.2151 Disc Loss: 0.0248321 Q Losses: [0.018848419, 0.19859406]
epoch:1 batch_done:58 Gen Loss: 12.8083 Disc Loss: 0.0211046 Q Losses: [0.025010642, 0.19081984]
epoch:1 batch_done:59 Gen Loss: 12.7208 Disc Loss: 0.000523382 Q Losses: [0.024675444, 0.21270972]
epoch:1 batch_done:60 Gen Loss: 11.4898 Disc Loss: 0.0005525 Q Losses: [0.017364506, 0.19549885]
epoch:1 batch_done:61 Gen 

epoch:1 batch_done:135 Gen Loss: 7.96727 Disc Loss: 0.000906117 Q Losses: [0.023441916, 0.13309869]
epoch:1 batch_done:136 Gen Loss: 10.9062 Disc Loss: 0.000424412 Q Losses: [0.012207425, 0.16570376]
epoch:1 batch_done:137 Gen Loss: 12.255 Disc Loss: 0.00140628 Q Losses: [0.015023682, 0.12041267]
epoch:1 batch_done:138 Gen Loss: 8.14157 Disc Loss: 0.00641435 Q Losses: [0.021257028, 0.12885933]
epoch:1 batch_done:139 Gen Loss: 10.9433 Disc Loss: 0.00279121 Q Losses: [0.012817391, 0.12873968]
epoch:1 batch_done:140 Gen Loss: 13.3846 Disc Loss: 0.00147113 Q Losses: [0.01973368, 0.13627192]
epoch:1 batch_done:141 Gen Loss: 10.0532 Disc Loss: 0.000958233 Q Losses: [0.014905505, 0.13417849]
epoch:1 batch_done:142 Gen Loss: 9.06075 Disc Loss: 0.000852468 Q Losses: [0.017854815, 0.12085256]
epoch:1 batch_done:143 Gen Loss: 6.98107 Disc Loss: 0.00309267 Q Losses: [0.019163763, 0.12073275]
epoch:1 batch_done:144 Gen Loss: 7.67384 Disc Loss: 0.00057834 Q Losses: [0.014495621, 0.1504655]
epoch:1 b

...

epoch:9 batch_done:163 Gen Loss: nan Disc Loss: nan Q Losses: [0.14994434, 2.3013353]
epoch:9 batch_done:164 Gen Loss: nan Disc Loss: nan Q Losses: [0.19107784, 2.3027587]
epoch:9 batch_done:165 Gen Loss: nan Disc Loss: nan Q Losses: [0.18583195, 2.3033643]
epoch:9 batch_done:166 Gen Loss: nan Disc Loss: nan Q Losses: [0.18792368, 2.3044629]
epoch:9 batch_done:167 Gen Loss: nan Disc Loss: nan Q Losses: [0.19095665, 2.2999966]
epoch:9 batch_done:168 Gen Loss: nan Disc Loss: nan Q Losses: [0.17991355, 2.3037391]
epoch:9 batch_done:169 Gen Loss: nan Disc Loss: nan Q Losses: [0.21810028, 2.3003197]
epoch:9 batch_done:170 Gen Loss: nan Disc Loss: nan Q Losses: [0.18768877, 2.3004956]
epoch:9 batch_done:171 Gen Loss: nan Disc Loss: nan Q Losses: [0.15069, 2.3018098]
epoch:9 batch_done:172 Gen Loss: nan Disc Loss: nan Q Losses: [0.16860631, 2.3032165]
epoch:9 batch_done:173 Gen Loss: nan Disc Loss: nan Q Losses: [0.16573709, 2.3009377]
epoch:9 batch_done:174 Gen Loss: nan Disc Loss: nan Q Los

epoch:10 batch_done:52 Gen Loss: nan Disc Loss: nan Q Losses: [0.16673706, 2.3040471]
epoch:10 batch_done:53 Gen Loss: nan Disc Loss: nan Q Losses: [0.15030131, 2.3019638]
epoch:10 batch_done:54 Gen Loss: nan Disc Loss: nan Q Losses: [0.17023194, 2.3033328]
epoch:10 batch_done:55 Gen Loss: nan Disc Loss: nan Q Losses: [0.15790305, 2.3043227]
epoch:10 batch_done:56 Gen Loss: nan Disc Loss: nan Q Losses: [0.15839465, 2.3015978]
epoch:10 batch_done:57 Gen Loss: nan Disc Loss: nan Q Losses: [0.18853232, 2.3024805]
epoch:10 batch_done:58 Gen Loss: nan Disc Loss: nan Q Losses: [0.14543091, 2.3040061]
epoch:10 batch_done:59 Gen Loss: nan Disc Loss: nan Q Losses: [0.14061508, 2.3030965]
epoch:10 batch_done:60 Gen Loss: nan Disc Loss: nan Q Losses: [0.16125154, 2.3023157]
epoch:10 batch_done:61 Gen Loss: nan Disc Loss: nan Q Losses: [0.16707444, 2.3003964]
epoch:10 batch_done:62 Gen Loss: nan Disc Loss: nan Q Losses: [0.16282839, 2.303483]
epoch:10 batch_done:63 Gen Loss: nan Disc Loss: nan Q L

epoch:10 batch_done:148 Gen Loss: nan Disc Loss: nan Q Losses: [0.17950176, 2.3032398]
epoch:10 batch_done:149 Gen Loss: nan Disc Loss: nan Q Losses: [0.1613051, 2.302428]
epoch:10 batch_done:150 Gen Loss: nan Disc Loss: nan Q Losses: [0.17886168, 2.3028679]
epoch:10 batch_done:151 Gen Loss: nan Disc Loss: nan Q Losses: [0.15416804, 2.3015242]
epoch:10 batch_done:152 Gen Loss: nan Disc Loss: nan Q Losses: [0.18672252, 2.3028781]
epoch:10 batch_done:153 Gen Loss: nan Disc Loss: nan Q Losses: [0.1611837, 2.3022702]
epoch:10 batch_done:154 Gen Loss: nan Disc Loss: nan Q Losses: [0.16070892, 2.3019598]
epoch:10 batch_done:155 Gen Loss: nan Disc Loss: nan Q Losses: [0.15984958, 2.3019295]
epoch:10 batch_done:156 Gen Loss: nan Disc Loss: nan Q Losses: [0.17115496, 2.3009977]
epoch:10 batch_done:157 Gen Loss: nan Disc Loss: nan Q Losses: [0.17987153, 2.3031089]
epoch:10 batch_done:158 Gen Loss: nan Disc Loss: nan Q Losses: [0.20884533, 2.3033586]
epoch:10 batch_done:159 Gen Loss: nan Disc Los

epoch:11 batch_done:36 Gen Loss: nan Disc Loss: nan Q Losses: [0.14699268, 2.3009071]
epoch:11 batch_done:37 Gen Loss: nan Disc Loss: nan Q Losses: [0.17827363, 2.3018484]
epoch:11 batch_done:38 Gen Loss: nan Disc Loss: nan Q Losses: [0.16100727, 2.3031259]
epoch:11 batch_done:39 Gen Loss: nan Disc Loss: nan Q Losses: [0.16628501, 2.3040385]
epoch:11 batch_done:40 Gen Loss: nan Disc Loss: nan Q Losses: [0.16883489, 2.3029149]
epoch:11 batch_done:41 Gen Loss: nan Disc Loss: nan Q Losses: [0.19426537, 2.3011332]
epoch:11 batch_done:42 Gen Loss: nan Disc Loss: nan Q Losses: [0.14268567, 2.301332]
epoch:11 batch_done:43 Gen Loss: nan Disc Loss: nan Q Losses: [0.16535132, 2.3028131]
epoch:11 batch_done:44 Gen Loss: nan Disc Loss: nan Q Losses: [0.17234345, 2.3027041]
epoch:11 batch_done:45 Gen Loss: nan Disc Loss: nan Q Losses: [0.14944983, 2.3032188]
epoch:11 batch_done:46 Gen Loss: nan Disc Loss: nan Q Losses: [0.14086767, 2.305618]
epoch:11 batch_done:47 Gen Loss: nan Disc Loss: nan Q Lo

epoch:11 batch_done:132 Gen Loss: nan Disc Loss: nan Q Losses: [0.18676773, 2.3019378]
epoch:11 batch_done:133 Gen Loss: nan Disc Loss: nan Q Losses: [0.19059274, 2.302151]
epoch:11 batch_done:134 Gen Loss: nan Disc Loss: nan Q Losses: [0.15178475, 2.3011532]
epoch:11 batch_done:135 Gen Loss: nan Disc Loss: nan Q Losses: [0.17691568, 2.3008964]
epoch:11 batch_done:136 Gen Loss: nan Disc Loss: nan Q Losses: [0.14319187, 2.3029506]
epoch:11 batch_done:137 Gen Loss: nan Disc Loss: nan Q Losses: [0.14940996, 2.303354]
epoch:11 batch_done:138 Gen Loss: nan Disc Loss: nan Q Losses: [0.13158064, 2.3031077]
epoch:11 batch_done:139 Gen Loss: nan Disc Loss: nan Q Losses: [0.16987012, 2.3021739]


## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 